In [ ]:
import pandas as pd

In [ ]:
need_to_label_via_classifier_path = "C:/Users/iliapl/Documents/CoronaVirusProject/repo/corona_healthcare_workers/manual_labeling/datasets/Classification_results/"

In [ ]:
target_field = 'Account_Type'
#target_field = 'Occupation_Type'

need_to_label_account_type_df = pd.read_csv(need_to_label_via_classifier_path + "unlabeled_predictions_{0}_using_description_and_SVM_classifier_balanced.csv".format(target_field))
need_to_label_account_type_df.head()

In [ ]:
#target_field = 'Account_Type'
target_field = 'Occupation_Type'

need_to_label_occupation_type_df = pd.read_csv(need_to_label_via_classifier_path + "unlabeled_predictions_{0}_using_description_and_SVM_classifier_balanced.csv".format(target_field))
need_to_label_occupation_type_df.head()

In [ ]:
confidence_range = (0.45, 0.55)

In [ ]:
need_to_label_account_type_df = need_to_label_account_type_df[(need_to_label_account_type_df["confidence_to_class_1"] > confidence_range[0]) & (need_to_label_account_type_df["confidence_to_class_1"] < confidence_range[1])]
print(need_to_label_account_type_df.shape)

need_to_label_occupation_type_df = need_to_label_occupation_type_df[(need_to_label_occupation_type_df["confidence_to_class_1"] > confidence_range[0]) & (need_to_label_occupation_type_df["confidence_to_class_1"] < confidence_range[1])]
print(need_to_label_occupation_type_df.shape)

In [ ]:
# get intersection between the two low confidence classifier results

need_to_label_df = need_to_label_account_type_df.merge(need_to_label_occupation_type_df, on=['author_screen_name', 'author_osn_id'])

In [ ]:
need_to_label_df.shape

In [ ]:
need_to_label_df.head()

In [ ]:
class Annotator():
    def __init__(self, aid):
        self._aid = aid
        self._users_to_review = []
        self._user_ids_to_review = []
        
    def get_aid(self):
        return self._aid
    
    def set_aid(self, aid):
        self._aid = aid
        
    def add_user(self, username):
        self._users_to_review.append(username)
        
        
    def add_user_id(self, user_id):
        self._user_ids_to_review.append(user_id)
        
    def get_users_to_review(self):
        return self._users_to_review
    
    def get_user_ids_to_review(self):
        return self._user_ids_to_review
    
    def get_users_count(self):
        return len(self._users_to_review)
    
    def __repr__(self):
        return "aid: {0} users_count: {1}".format(self._aid, len(self._users_to_review))

In [ ]:
iteration_number = 20

annotator_count = 3
annotator_dict = {}
for i in range(annotator_count):
    annotator = Annotator(i)
    aid = annotator.get_aid()
    annotator_dict[aid] = annotator 
annotator_dict

In [ ]:
def get_min_count_for_annotator():
    annotators = annotator_dict.values()
    assigned_users = [annotator.get_users_count() for annotator in annotators]
    min_assigned_users = min(assigned_users)
    return min_assigned_users


def get_aid_to_assign_user(min_assigned_users):
    annotators = annotator_dict.values()
    annotator_to_assign = [annotator.get_aid() for annotator in annotators 
                           if annotator.get_users_count() == min_assigned_users][0]
    return annotator_to_assign

def assign_user_to_annotator(username, user_id):
    min_assigned_users = get_min_count_for_annotator()
    aid = get_aid_to_assign_user(min_assigned_users)
    
    annotator_dict[aid].add_user(username)
    annotator_dict[aid].add_user_id(user_id)

In [ ]:
#df = pd.read_csv("/home/aviade/HCW_Corona_TWINT/all_tweets.csv", header=None, delimiter='\t')
path = "C:/Users/iliapl/Documents/CoronaVirusProject/repo/corona_healthcare_workers/manual_labeling/datasets/"
file_name = "top40percent_71972_authors.csv"
df = pd.read_csv(path + file_name)
df.head()

In [ ]:
import random 

random_users_count = 90
need_to_label_df = need_to_label_df.sample(n=random_users_count)

for i, row in need_to_label_df.iterrows():
    username = row["author_screen_name"]
    user_id = row["author_osn_id"]
    assign_user_to_annotator(username, user_id)
    assign_user_to_annotator(username, user_id)

In [ ]:
import os

output_path = "C:/Users/iliapl/Documents/CoronaVirusProject/repo/corona_healthcare_workers/manual_labeling/datasets/Round_{0}/".format(iteration_number)
if not os.path.exists(output_path):
    os.mkdir(output_path)

annotators = annotator_dict.values()
for annotator in annotators:
    aid = annotator.get_aid()
    
    users = annotator.get_users_to_review()
    users_series = pd.Series(users)
    
    user_ids = annotator.get_user_ids_to_review()
    user_ids_series = pd.Series(user_ids)
    
    annotator_df = pd.DataFrame(users_series, columns = ['username'])
    annotator_df["author_osn_id"] = user_ids_series
    annotator_df["url"] = annotator_df["username"].apply(lambda x: "https://twitter.com/" + str(x))
    annotator_df["Account_Type (Individual/Organization/Other)"] = ""
    annotator_df["Occupation_Type (HCP/Not HCP)"] = ""
    annotator_df["Comments"] = ""
    
    
    annotator_df.to_csv(output_path + f"annotator_{aid}_users_to_label_round_{iteration_number}.csv", index=False)